In [31]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

class FertilizerStandardizer:
    def __init__(self):
        self.icv_fertilizer_list = ["Solution Liquide N 39"]
        self.default_mineral_density = 1.325
        self.organic_solid_density = 1000
        self.organic_liquid_density = 1.0
        self.type_mapping = {
            "Fertilisation et amendement organique": "engrais organiques",
            "Fertilisation et amendement mineral - foliaire inclus": "engrais mineraux",
            "Fertilisation organique": "engrais organiques",
            "Fertilisation minérale": "engrais mineraux"
        }

    def standardize_fertilizer_name(self, name):
        if pd.isna(name): return name
        if isinstance(name, (pd.Timestamp, datetime)):
            name = name.strftime('%d-%m-%Y')
        name = str(name).upper()
        name = re.sub(r'[^\w\s\-]', ' ', name)
        name = re.sub(r'\s+', ' ', name)
        return name.strip()

    def is_in_icv_list(self, fertilizer_name):
        std_name = self.standardize_fertilizer_name(fertilizer_name)
        std_icv_list = [self.standardize_fertilizer_name(name) for name in self.icv_fertilizer_list]
        return std_name in std_icv_list

    def standardize_type_engrais(self, intervention_type):
        if pd.isna(intervention_type):
            return "engrais mineraux"
        type_std = self.type_mapping.get(intervention_type)
        if type_std is None:
            print(f"Avertissement: Type d'intervention non reconnu: {intervention_type}")
            return "engrais mineraux"
        return type_std

    def convert_to_kg(self, row):
        try:
            if pd.isna(row.get('unite_intrant_intervention')) or pd.isna(row.get('quantite_totale')):
                return np.nan
            unite = str(row.get('unite_intrant_intervention', '')).upper()
            type_engrais = str(row.get('type_engrais', '')).lower()
            quantite = float(row.get('quantite_totale', 0))
            
            if unite == 'KG':
                return quantite
            elif unite == 'L':
                return quantite * (self.organic_liquid_density if 'organique' in type_engrais else self.default_mineral_density)
            elif unite == 'M3':
                if 'organique' in type_engrais:
                    return quantite * self.organic_solid_density
                return quantite * self.default_mineral_density * 1000
            else:
                print(f"Avertissement: Unité non reconnue: {unite} pour l'engrais {row.get('libelle', 'Unknown')}")
                return np.nan
        except Exception as e:
            print(f"Erreur lors de la conversion pour l'engrais {row.get('libelle', 'Unknown')}: {str(e)}")
            return np.nan

    def convert_dose_to_kg_ha(self, row):
        try:
            if pd.isna(row.get('unite')) or pd.isna(row.get('dose')):
                return np.nan
            
            unite = str(row['unite']).upper()
            if unite == 'KG/HA':
                return row['dose']
            elif unite == 'T/HA':
                return row['dose'] * 1000
            elif unite == 'L/HA':
                if row['type_engrais'] == 'engrais organiques':
                    return row['dose'] * self.organic_liquid_density
                return row['dose'] * self.default_mineral_density
            elif unite == 'M3/HA':
                if row['type_engrais'] == 'engrais organiques':
                    return row['dose'] * self.organic_solid_density
                return row['dose'] * self.default_mineral_density * 1000
            else:
                print(f"Avertissement: Unité non reconnue: {unite} pour l'engrais {row['libelle']}")
                return np.nan
        except Exception as e:
            print(f"Erreur lors de la conversion pour l'engrais {row['libelle']}: {str(e)}")
            return np.nan

    def calculate_nutrients(self, row):
        try:
            if not row['in_icv_list']:
                if row['type_engrais'] == 'engrais organiques':
                    volume_m3 = row['quantite_totale'] if row['unite_intrant_intervention'].upper() == 'M3' \
                        else row['quantite_kg'] / self.organic_solid_density
                    
                    row['N_kg_ha'] = row['fertilisant.composition.n_total'] * volume_m3 / row['surface_travaillee_ha']
                    row['P_kg_ha'] = row['fertilisant.composition.p'] * volume_m3 / row['surface_travaillee_ha']
                    row['K_kg_ha'] = row['fertilisant.composition.k'] * volume_m3 / row['surface_travaillee_ha']
                else:
                    row['N_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.n_total'] / 100
                    row['P_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.p'] / 100
                    row['K_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.k'] / 100
            return row
        except Exception as e:
            print(f"Erreur lors du calcul des nutriments pour l'engrais {row['libelle']}: {str(e)}")
            return row

    def standardize_mesparcelles(self, data, aggregate=False):
        """Standardise les données MesParcelles"""
        print("Standardisation des données MesParcelles...")
        
        fertilisation_types = [
            "Fertilisation et amendement organique",
            "Fertilisation et amendement mineral - foliaire inclus"
        ]
        df = data[data['type_intervention.libelle'].isin(fertilisation_types)].copy()
        
        if 'culture.libelle' in df.columns:
            df = df.rename(columns={'culture.libelle': 'culture'})
            print("Colonne culture.libelle trouvée et renommée en 'culture'")
        else:
            print("ATTENTION: culture.libelle non trouvée!")
            print("Colonnes disponibles:", df.columns.tolist())
            return None
            
        df['type_engrais'] = df['type_intervention.libelle'].apply(self.standardize_type_engrais)
        df['in_icv_list'] = df['libelle'].apply(self.is_in_icv_list)
        df['quantite_kg'] = df.apply(self.convert_to_kg, axis=1)
        df['dose_kg_ha'] = df['quantite_kg'] / df['surface_travaillee_ha']
        df = df.apply(self.calculate_nutrients, axis=1)
        
        if aggregate:
            return self._aggregate_by_exploitation_and_culture(df)
        return df

    def standardize_smag(self, data, cultures_data, aggregate=False):
        """Standardise les données SMAG"""
        print("\nStandardisation des données SMAG...")
        
        # Filtrer les fertilisations
        fertilisation_types = ["Fertilisation organique", "Fertilisation minérale"]
        df = data[data["Type d'intervention"].isin(fertilisation_types)].copy()
        
        # Renommage initial des colonnes
        base_columns = {
        'Intrant': 'libelle',
        'Unité': 'unite',
        'Dose': 'dose',
        'SIRET': 'siret_exploitation',
        'Nom de la parcelle': 'uuid_parcelle',
        'Code edi parcelle': 'code_edi',
        'Surface intervention sur parcelle': 'Surface intervention sur parcelle',  # Ajout ici
        'N': 'N_kg_ha',
        'P': 'P_kg_ha',
        'K': 'K_kg_ha'
        }   
        df = df.rename(columns=base_columns)
        
        # Assurons-nous que les colonnes sont du même type
        df['siret_exploitation'] = df['siret_exploitation'].astype(str)
        df['uuid_parcelle'] = df['uuid_parcelle'].astype(str)
        df['code_edi'] = df['code_edi'].astype(str)
        
        cultures_data['SIRET'] = cultures_data['SIRET'].astype(str)
        cultures_data['Nom de la parcelle'] = cultures_data['Nom de la parcelle'].astype(str)
        cultures_data['Code edi parcelle'] = cultures_data['Code edi parcelle'].astype(str)
        
        # Fusion avec cultures
        df = pd.merge(
            df,
            cultures_data[['SIRET', 'Nom de la parcelle', 'Code edi parcelle', 'Culture']],
            left_on=['siret_exploitation', 'uuid_parcelle', 'code_edi'],
            right_on=['SIRET', 'Nom de la parcelle', 'Code edi parcelle'],
            how='left'
        )
        
        # Nettoyage et renommage final
        df = df.rename(columns={'Culture': 'culture'})
        # On supprime les colonnes redondantes après la jointure
        df = df.drop(['SIRET', 'Nom de la parcelle', 'Code edi parcelle'], axis=1, errors='ignore')
        
        # Standardisation
        df['type_engrais'] = df["Type d'intervention"].apply(self.standardize_type_engrais)
        df['in_icv_list'] = df['libelle'].apply(self.is_in_icv_list)
        df.loc[df['in_icv_list'], 'dose_kg_ha'] = df[df['in_icv_list']].apply(
            self.convert_dose_to_kg_ha, axis=1
        )
        
        # Log pour vérifier si la jointure a bien fonctionné
        total_rows = len(df)
        matched_rows = df['culture'].notna().sum()
        print(f"\nStatistiques de la jointure:")
        print(f"Nombre total de lignes: {total_rows}")
        print(f"Lignes avec une culture correspondante: {matched_rows}")
        print(f"Lignes sans correspondance: {total_rows - matched_rows}")
        
        return df if not aggregate else self._aggregate_by_exploitation_and_culture(df)

    def _aggregate_by_exploitation_and_culture(self, df, smag_data, mesparcelles_data):
        """
        df: données déjà standardisées (format final avec toutes les colonnes)
        smag_data: fichier SMAG original pour récupérer les surfaces
        mesparcelles_data: fichier MP original pour récupérer les surfaces
        """
        try:
            # Séparation SMAG et MP
            df_smag = df[df['source'] == 'smag'].copy()
            df_mp = df[df['source'] == 'mesparcelles'].copy()
            
            # 1. Récupération des surfaces pour SMAG
            if not df_smag.empty:
                smag_surfaces = smag_data[
                    ['SIRET', 'Nom de la parcelle', 'Code edi parcelle', 'Surface intervention sur parcelle']
                ].rename(columns={
                    'SIRET': 'siret_exploitation',
                    'Surface intervention sur parcelle': 'surface'
                })
                
                # Conversion en string pour la jointure
                smag_surfaces['siret_exploitation'] = smag_surfaces['siret_exploitation'].astype(str)
                df_smag['siret_exploitation'] = df_smag['siret_exploitation'].astype(str)
                smag_surfaces['Code edi parcelle'] = smag_surfaces['Code edi parcelle'].astype(str)
                df_smag['code_edi'] = df_smag['code_edi'].astype(str)
                
                df_smag = pd.merge(
                    df_smag,
                    smag_surfaces,
                    left_on=['siret_exploitation', 'uuid_parcelle', 'code_edi'],
                    right_on=['siret_exploitation', 'Nom de la parcelle', 'Code edi parcelle'],
                    how='left'
                )
            
            # 2. Récupération des surfaces pour MP
            if not df_mp.empty:
                mp_surfaces = mesparcelles_data[
                    ['uuid_intervention', 'surface_travaillee_ha']
                ].rename(columns={
                    'surface_travaillee_ha': 'surface'
                })
                
                # Conversion en string pour uuid_intervention
                mp_surfaces['uuid_intervention'] = mp_surfaces['uuid_intervention'].astype(str)
                df_mp['uuid_intervention'] = df_mp['uuid_intervention'].astype(str)
                
                df_mp = pd.merge(
                    df_mp,
                    mp_surfaces,
                    on='uuid_intervention',
                    how='left'
                )
            
            # 3. Fonction d'agrégation avec moyenne pondérée
            def weighted_mean(group):
                result = pd.Series()
                
                # Pour tous les engrais ICV
                dose_cols = [col for col in group.columns if col.endswith('_dose')]
                for col in dose_cols:
                    if 'surface' in group.columns:
                        # Filtrer les NaN avant de faire la moyenne
                        mask = ~group[col].isna()
                        if mask.any():  # s'il reste des valeurs après filtrage des NaN
                            result[col] = np.average(
                                group.loc[mask, col],
                                weights=group.loc[mask, 'surface']
                            )
                        else:
                            result[col] = np.nan
                
                # Pour tous les nutriments
                nutrient_cols = [
                    'N_organique', 'P_organique', 'K_organique',
                    'N_mineral', 'P_mineral', 'K_mineral'
                ]
                for col in nutrient_cols:
                    if 'surface' in group.columns:
                        # Filtrer les NaN avant de faire la moyenne
                        mask = ~group[col].isna()
                        if mask.any():
                            result[col] = np.average(
                                group.loc[mask, col],
                                weights=group.loc[mask, 'surface']
                            )
                        else:
                            result[col] = np.nan
                
                return result
            
            # 4. Agrégation séparée pour SMAG et MP
            results = []
            
            if not df_smag.empty and 'surface' in df_smag.columns:
                smag_agg = df_smag.groupby(['siret_exploitation', 'culture']).apply(weighted_mean)
                smag_agg['source'] = 'smag'
                results.append(smag_agg)
            
            if not df_mp.empty and 'surface' in df_mp.columns:
                mp_agg = df_mp.groupby(['siret_exploitation', 'culture']).apply(weighted_mean)
                mp_agg['source'] = 'mesparcelles'
                results.append(mp_agg)
            
            # 5. Combinaison des résultats
            if results:
                final_result = pd.concat(results)
                return final_result.reset_index()
            else:
                print("ATTENTION: Aucun résultat d'agrégation!")
                return None
                
        except Exception as e:
            print(f"Erreur lors de l'agrégation: {str(e)}")
            print(f"Type d'erreur: {type(e)}")
            return None
    def standardize_columns(self, df):
        print("Colonnes disponibles avant standardisation:", df.columns.tolist())
        
        try:
            # Définir l'ordre des colonnes d'abord
            columns_order = [
                'uuid_intervention',
                'siret_exploitation', 
                'uuid_parcelle',
                'code_edi',
                'culture',
                'source',
                'Surface intervention sur parcelle',    
                'surface_travaillee_ha',               
                *[f"{engrais}_dose" for engrais in self.icv_fertilizer_list],  
                'N_organique', 'P_organique', 'K_organique',
                'N_mineral', 'P_mineral', 'K_mineral'
            ]
            
            # Pour les doses ICV
            icv_doses = df[df['in_icv_list']].pivot(
                index=['siret_exploitation', 'uuid_parcelle', 'culture', 'source', 'code_edi', 'uuid_intervention'],
                columns='libelle',
                values='dose_kg_ha'
            ).add_suffix('_dose').reset_index()

            # Pour les nutriments organiques
            df_org = df[~df['in_icv_list'] & (df['type_engrais'] == 'engrais organiques')].copy()
            df_org = df_org.rename(columns={
                'N_kg_ha': 'N_organique',
                'P_kg_ha': 'P_organique',
                'K_kg_ha': 'K_organique'
            })

            # Pour les nutriments minéraux
            df_min = df[~df['in_icv_list'] & (df['type_engrais'] == 'engrais mineraux')].copy()
            df_min = df_min.rename(columns={
                'N_kg_ha': 'N_mineral',
                'P_kg_ha': 'P_mineral',
                'K_kg_ha': 'K_mineral'
            })

            # Garder les colonnes nécessaires et dans le bon ordre
            result = pd.concat([icv_doses, df_org, df_min])
            
            # S'assurer que toutes les colonnes existent
            for col in columns_order:
                if col not in result.columns:
                    result[col] = None
                    
            result = result[columns_order]
            
            return result

        except Exception as e:
            print(f"Erreur lors de la transformation: {str(e)}")
            print("État des données:")
            print(df.head())
            return df
    def combine_sources(self, mesparcelles_data, smag_data, smag_cultures_data):
        print("\nTraitement des données MesParcelles...")
        mp_std = self.standardize_mesparcelles(mesparcelles_data, aggregate=False)
        
        print("\nTraitement des données SMAG...")
        smag_std = self.standardize_smag(smag_data, smag_cultures_data, aggregate=False)
        
        if mp_std is None or smag_std is None:
            print("ERREUR: problème dans la standardisation des données")
            return None, None
        
        # Ajout de la source avant la standardisation des colonnes
        mp_std['source'] = 'mesparcelles'
        smag_std['source'] = 'smag'
        
        # Combiner avant de standardiser les colonnes
        combined_raw = pd.concat([mp_std, smag_std], ignore_index=True)
        
        # Standardiser les colonnes des données combinées
        standardized_raw = self.standardize_columns(combined_raw)

        # Faire l'agrégation sur les données standardisées (pas les raw)
        combined_aggregated = self._aggregate_by_exploitation_and_culture(
        standardized_raw, 
        smag_data,
        mesparcelles_data
        )
        
        return standardized_raw, combined_aggregated

# Code d'utilisation
if __name__ == "__main__":
    print("\nImport des données...")
    
    mp_path = r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\mesparcelles_data.xlsx"
    smag_path = r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\SMAG.xlsx"
    smag_cultures_path = r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\SMAG_cultures.xlsx"
    
    mp_data = pd.read_excel(mp_path, sheet_name='Intervention_Details')
    smag_data = pd.read_excel(smag_path)
    smag_cultures = pd.read_excel(smag_cultures_path)
    
    standardizer = FertilizerStandardizer()
    raw_data, aggregated_data = standardizer.combine_sources(mp_data, smag_data, smag_cultures)
    
    if raw_data is not None and aggregated_data is not None:
        # Export dans un seul fichier Excel avec deux feuilles
        output_path = 'resultats_standardisation.xlsx'
        with pd.ExcelWriter(output_path) as writer:
            raw_data.to_excel(writer, sheet_name='Données brutes', index=False)
            aggregated_data.to_excel(writer, sheet_name='Données agrégées', index=False)
        print(f"\nRésultats exportés dans '{output_path}'")
        
        # Statistiques
        print("\nStatistiques données brutes:")
        print(f"Nombre total de parcelles: {len(raw_data)}")
        print(f"Nombre d'exploitations: {raw_data['siret_exploitation'].nunique()}")
        print(f"Nombre de cultures: {raw_data['culture'].nunique()}")
        
        
        print("\nStatistiques données agrégées:")
        print(f"Nombre de combinaisons exploitation-culture: {len(aggregated_data)}")
        print(f"Nombre d'exploitations: {aggregated_data['siret_exploitation'].nunique()}")
        print(f"Nombre de cultures: {aggregated_data['culture'].nunique()}")
        


Import des données...

Traitement des données MesParcelles...
Standardisation des données MesParcelles...
Colonne culture.libelle trouvée et renommée en 'culture'

Traitement des données SMAG...

Standardisation des données SMAG...

Statistiques de la jointure:
Nombre total de lignes: 19575
Lignes avec une culture correspondante: 19575
Lignes sans correspondance: 0
Colonnes disponibles avant standardisation: ['K_kg_ha', 'N_kg_ha', 'P_kg_ha', 'culture', 'culture.id_culture', 'date_debut', 'date_fin', 'dose_kg_ha', 'fertilisant', 'fertilisant.composition.cao', 'fertilisant.composition.k', 'fertilisant.composition.mgo', 'fertilisant.composition.n_total', 'fertilisant.composition.p', 'fertilisant.composition.s', 'fertilisant.composition.unite', 'fertilisant.condition_epandage', 'fertilisant.condition_epandage.id_condition_epandage', 'fertilisant.condition_epandage.libelle', 'id_intrant', 'in_icv_list', 'libelle', 'link_parcelle', 'materiels', 'numero_lot', 'phyto', 'phyto.cible', 'phyto.c

C:\Users\MonirNajem\AppData\Local\Temp\ipykernel_11352\2838888170.py:294: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  smag_agg = df_smag.groupby(['siret_exploitation', 'culture']).apply(weighted_mean)
C:\Users\MonirNajem\AppData\Local\Temp\ipykernel_11352\2838888170.py:299: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mp_agg = df_mp.groupby(['siret_exploitation', 'culture']).apply(weighted_mean)



Résultats exportés dans 'resultats_standardisation.xlsx'

Statistiques données brutes:
Nombre total de parcelles: 19583
Nombre d'exploitations: 202
Nombre de cultures: 57

Statistiques données agrégées:
Nombre de combinaisons exploitation-culture: 646
Nombre d'exploitations: 202
Nombre de cultures: 57
